In [ ]:
!pip install -r requirements.txt

In [ ]:
%%writefile .env
GOOGLE_API_KEY = "paste your google api key"

In [ ]:

import streamlit as st
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as genai
from langchain_community.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv

load_dotenv()

genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

def get_pdf_text(pdf_docs):
  text=""
  for pdf in pdf_docs:
    pdf_reader=PdfReader(pdf)
    for page in pdf_reader.pages:
      text+=page.extract_text()
  return text
def get_text_chunks(text):
  text_splitter=RecursiveCharacterTextSplitter(chunk_size=10000,chunk_overlap=200)
  chunks = text_splitter.split_text(text)
  return chunks

def  get_vector_store(text_chunks):
  embeddings=GoogleGenerativeAIEmbeddings(model="models/embedding-001")
  vector_store=FAISS.from_texts(embeddings,text_chunks)
  vector_store.save_local("faiss_index")

def get_conversation_chain():
  prompt_template="""
  Answer the question as detailed as possible from the provided context, make sure to provide all the details,if the answer is not in
  provided context just say, "answer is not available in the context", don't provide the wrong answer\n\n
  context:\n {context}?\n
  Question:\n {question}?\n

  Answer:
  """

  model = ChatGoogleGenerativeAI(model="gemini-pro",temperature=0.3)
  prompt = PromptTemplate(template=prompt_template,input_variables=["context","question"])
  llm=ChatGoogleGenerativeAI(model="models/chat-bison-001")
  chain = load_qa_chain(llm,chain_type="stuff",prompt=prompt)
  return chain

def user_input(user_question):
  embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

  new_db = FAISS.load_local("faiss_index",embeddings)

  docs = new_db.similarity_search(user_question)

  chain = get_conversation_chain()

  response = chain(
      {"input_documents":docs,"question":user_question}
      , return_only_outputs=True)
  print(response)
  st.write("Reply: ",response["output_text"])

def main():
  st.set_page_config("Chat with Multiple PDF")
  st.header("chat with PDF using Gemini")

  user_question = st.text_input("answer a question from the pdf")

  if user_question:
    user_input(user_question)

  with st.sidebar:
    st.subheader("Menu:")
    pdf_docs = st.file_uploader("Upload your pdfs here and click on 'Process'",accept_multiple_files=True)
    if st.button("Submit & Process"):
      with st.spinner("Processing ..."):
        raw_text = get_pdf_text(pdf_docs)
        text_chunks = get_text_chunks(raw_text)
        get_vector_store(text_chunks)
        st.success("done")

if __name__ == "__main__":
  main()

In [ ]:
!npm install -g localtunnel

In [ ]:
!curl ipv4.icanhazip.com

In [ ]:
!streamlit run /content/chatwithpdf.py &> ./logs.txt & npx localtunnel --port 8501